Imports

In [1]:
import joblib
import numpy as np
import tensorflow as tf
from collections import Counter
import random

from tensorflow.keras import datasets, layers, models

import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt

# Change matplotlib to use a Japanese font
# to actually be able to display correctly
matplotlib.rcParams.update(
    {
        'text.usetex': False,
        'font.family': 'MS Gothic'
    }
)

Global variables

In [2]:
debug = False
num_chars = 100

Load data, print some for verification

In [3]:
#data = joblib.load(f'{base_name}_{width}x{width}px.pkl.part1')
data = joblib.load('./Input/all_characters_64x64px.pkl.part1')

if debug:
    print('Number of samples: ', len(data['data']))
    print('keys: ', list(data.keys()))
    print('description: ', data['description'])
    print('image shape: ', data['data'][0].shape)
    print('labels:', np.unique(data['label']))

    #Counter(data['label'])


    plt.figure(figsize=(10,10))
    for i in range(25):
        index = random.randint(0, len(data['data']) - 1)
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(data['data'][index], cmap=plt.cm.binary)
        # The CIFAR labels happen to be arrays, 
        # which is why you need the extra index
        plt.xlabel(data['label'][index])
    plt.show()


Split data into training and test

In [4]:
from sklearn.model_selection import train_test_split

#X = np.array(data['data'])
#y = np.array(data['label'])

n_samples = len(data['data'])


X_train, X_test, y_train, y_test = train_test_split(
    np.array(data['data'][:num_chars*200]),
    np.array(data['label'][:num_chars*200]),
    test_size=0.4,
    shuffle=True,
    random_state=42)

del data

X_train = X_train.reshape(X_train.shape[0], 64, 64, 1)
X_test = X_test.reshape(X_test.shape[0], 64, 64, 1)




Change labels from strings to numbers so tensorflow can use them

In [5]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

Create the model

In [6]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())

model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(4096, activation='relu'))
#model.add(layers.Dense(4096, activation='relu'))
#model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(2048, activation='relu'))
model.add(layers.Dense(2048, activation='relu'))

model.add(layers.Dense(759))

if debug:
    model.summary()

Train the model

In [ ]:

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=30, 
                    validation_data=(X_test, y_test))

Display model information

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.0, 1.0])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print(test_acc)

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = y_test
 
#y_pred = le.inverse_transform(y_pred)
#y_true = le.inverse_transform(y_test)

labels = np.unique(le.inverse_transform(y_test))
print(labels)

conf_mtx = tf.math.confusion_matrix(y_true, y_pred)
plt.figure(figsize=(100, 80))
sns.heatmap(conf_mtx, xticklabels=labels, yticklabels=labels, 
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()
